In [ ]:
%matplotlib inline
import matplotlib as mpl
import numpy as np
from numpy import array, savetxt
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


In [ ]:
#carrega o dataset
dados = pd.read_csv('/home/erasmor/Downloads/2017/todos_apenas_baixa_representatividade.csv',sep=",",encoding = 'utf-8',  header=0,na_values='.',dtype={'Label':'category'})

#remove valores infinitos
dados.replace(-np.Inf, np.nan)

#substitui valores NaN
dados.fillna(dados.mean())


In [ ]:
dados.dtypes

In [ ]:
dados.memory_usage(deep=True)

In [ ]:
# verifica quantas instâncias (linhas) e quantos atributos (colunas) a base de dados contém
print("numero de linhas e colunas: ",dados.shape)

In [ ]:
#visualizar distribições por classes contidas no csv - informar nome da classe alvo
print(dados.groupby('Label').size())

In [ ]:
#X_raw_normalize = MinMaxScaler(X_raw_normalize.reshape(0, 1)).reshape(len(X_raw_normalize))
#X_raw_normalizetd2 = (X_raw_normalize - X_raw_normalize.min(axis=0)) / (X_raw_normalize.max(axis=0) - X_raw_normalize.min(axis=0))
# Obtendo os nomes das colunas do DataFrame como uma lista.
cols = list(dados.columns)
# colunas que nao serao normalizadas
cols.remove('Label')

# Copiando os dados e aplicando a normalizacao por reescala nas colunas do DataFrame que contem
# valores continuos. Por padrao, o metodo minmax_scale reescala com min=0 e max=1.
dados = dados[~dados.isin([np.nan, np.inf, -np.inf]).any(1)]
dados[cols] = dados[cols].apply(minmax_scale)


In [ ]:
#define as colunas de atributos e a coluna da classe (de 0 a 72 são atributos e após a 72 é a classe)
# "X_raw" é features/atributos e "y_raw" é target/classe ==> As duas formas abaixo dão certo.
#array = dataset.values
#X_raw = array[:,0:72]
#y_raw = array[:,72]
X_raw = dados.iloc[:, :-1].values # atributos
y_raw = dados.iloc[:, 78].values # classe de ataques
X_raw = np.nan_to_num(X_raw.astype(np.float32))


In [ ]:
#transformar a variável Y com valores categóricos das classses de ataques em valores:
labelencoder_y = LabelEncoder()
y_raw = labelencoder_y.fit_transform(y_raw)


In [ ]:
# Instanciando um PCA. O parametro n_components indica a quantidade de dimensoes que a base
# original sera reduzida.
pca = PCA(n_components=10, whiten=True,random_state=42)

In [ ]:
# Aplicando o pca na base de dados. O atributo 'values' retorna um numpy.array
# de duas dimensões (matriz) contendo apenas os valores numericos do DataFrame.
X_raw = pca.fit_transform(X_raw)

In [ ]:
def expected_error_reduction_function_knn(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_knn = ActiveLearner(estimator=KNeighborsClassifier(n_neighbors=5),X_training=X_train, y_training=y_train)
    arquivo_performance_knn = open("expected_error_reduction_performance_knn_dobra_"+indica_pool+".txt","a")
    arquivo_history_knn = ("expected_error_reduction_history_knn_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
    unqueried_score_knn = learner_knn.score(X_teste, y_teste)
    predictions = learner_knn.predict(X_teste)
    performance_history_knn.append(unqueried_score_knn)
    
    #inicio aprendizado ativo
 
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_knn, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_knn.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_knn.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
        
            new_score_knn = learner_knn.score(X_teste, y_teste)
            performance_history_knn.append(new_score_knn)
            clear_output(wait=True)
            print ('Accuracy KNN after query no. %d: %f' % (index+1, new_score_knn))
            arquivo_performance_knn.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_knn))
            predictions = learner_knn.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_knn.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_knn.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_knn.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_knn.write('======================================== \n')
                      
            
    arquivo_performance_knn.write("\n Avaliação final KNN \n")
    arquivo_performance_knn.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_knn, performance_history_knn,delimiter=",")
    arquivo_performance_knn.close()

def expected_error_reduction_function_rf(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_rf = ActiveLearner(estimator=RandomForestClassifier(random_state=42),X_training=X_train, y_training=y_train)
    arquivo_performance_rf = open("expected_error_reduction_performance_rf_dobra_"+indica_pool+".txt","a")
    arquivo_history_rf = ("expected_error_reduction_history_rf_dobra_"+indica_pool+".csv")
    
    #verifica a performance inicial
    unqueried_score_rf = learner_rf.score(X_teste, y_teste)
    predictions = learner_rf.predict(X_teste)
    performance_history_rf.append(unqueried_score_rf)
    
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_rf, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_rf.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_rf.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
        
            new_score_rf = learner_rf.score(X_teste, y_teste)
            performance_history_rf.append(new_score_rf)
            clear_output(wait=True)
            print ('Accuracy RF after query no. %d: %f' % (index+1, new_score_rf))
            arquivo_performance_rf.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_rf))
            predictions = learner_rf.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_rf.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_rf.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_rf.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_rf.write('======================================== \n')
            
            
    arquivo_performance_rf.write("\n Avaliação final RF \n")
    arquivo_performance_rf.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_rf, performance_history_rf,delimiter=",")
    arquivo_performance_rf.close()
    
def expected_error_reduction_function_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_tree =ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train, y_training=y_train)
    arquivo_performance_tree = open("expected_error_reduction_performance_tree_dobra_"+indica_pool+".txt","a")
    arquivo_history_tree = ("expected_error_reduction_history_tree_dobra_"+indica_pool+".csv")
    
    #verifica a performance inicial
    unqueried_score_tree = learner_tree.score(X_teste, y_teste)
    predictions = learner_tree.predict(X_teste)
    performance_history_tree.append(unqueried_score_tree)
    
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_tree, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_tree.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_tree.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
            new_score_tree = learner_tree.score(X_teste, y_teste)
            performance_history_tree.append(new_score_tree)
            clear_output(wait=True)
            print ('Accuracy TREE after query no. %d: %f' % (index+1, new_score_tree))
            arquivo_performance_tree.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_tree))
            predictions = learner_tree.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_tree.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_tree.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_tree.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_tree.write('======================================== \n')
            
            
    arquivo_performance_tree.write("\n Avaliação final TREE \n")
    arquivo_performance_tree.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_tree, performance_history_tree,delimiter=",")
    arquivo_performance_tree.close()

def expected_error_reduction_function_mlp(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_mlp = ActiveLearner(estimator=MLPClassifier(max_iter = 2000),X_training=X_train, y_training=y_train)
    arquivo_performance_mlp = open("expected_error_reduction_performance_mlp_dobra_"+indica_pool+".txt","a")
    arquivo_history_mlp = ("expected_error_reduction_history_mlp_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
    unqueried_score_mlp = learner_mlp.score(X_teste, y_teste)
    predictions = learner_mlp.predict(X_teste)
    performance_history_mlp.append(unqueried_score_mlp)
    
  
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_mlp, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_mlp.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_mlp.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
            new_score_mlp = learner_mlp.score(X_teste, y_teste)
            performance_history_mlp.append(new_score_mlp)
            clear_output(wait=True)
            print ('Accuracy MLP after query no. %d: %f' % (index+1, new_score_mlp))
            arquivo_performance_mlp.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_mlp))
            predictions = learner_mlp.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_mlp.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_mlp.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_mlp.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_mlp.write('======================================== \n')
            
            
    arquivo_performance_mlp.write("\n Avaliação final MLP \n")
    arquivo_performance_mlp.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_mlp, performance_history_mlp,delimiter=",")
    arquivo_performance_mlp.close()

def expected_error_reduction_function_xgb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from xgboost import XGBClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_xgb = ActiveLearner(estimator=GradientBoostingClassifier(n_estimators=7, learning_rate=1.0,max_depth=1, random_state=42),X_training=X_train, y_training=y_train)
    arquivo_performance_xgb = open("expected_error_reduction_performance_xgb_dobra_"+indica_pool+".txt","a")
    arquivo_history_xgb = ("expected_error_reduction_history_xgb_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
    unqueried_score_xgb = learner_xgb.score(X_teste, y_teste)
    predictions = learner_xgb.predict(X_teste)
    performance_history_xgb.append(unqueried_score_xgb)
    
  
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_xgb, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_xgb.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_xgb.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
            new_score_xgb = learner_xgb.score(X_teste, y_teste)
            performance_history_xgb.append(new_score_xgb)
            clear_output(wait=True)
            print ('Accuracy XGB after query no. %d: %f' % (index+1, new_score_xgb))
            arquivo_performance_xgb.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_xgb))
            predictions = learner_xgb.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_xgb.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_xgb.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_xgb.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_xgb.write('======================================== \n')
            
            
    arquivo_performance_xgb.write("\n Avaliação final XGB \n")
    arquivo_performance_xgb.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_xgb, performance_history_xgb,delimiter=",")
    arquivo_performance_xgb.close()

def expected_error_reduction_function_svm(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn import svm
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_svm = ActiveLearner(estimator=svm.SVC(kernel='linear',probability=True),X_training=X_train, y_training=y_train)
    arquivo_performance_svm = open("expected_error_reduction_performance_svm_dobra_"+indica_pool+".txt","a")
    arquivo_history_svm = ("expected_error_reduction_history_svm_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
    unqueried_score_svm = learner_svm.score(X_teste, y_teste)
    predictions = learner_svm.predict(X_teste)
    performance_history_svm.append(unqueried_score_svm)
    
  
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_svm, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_svm.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_svm.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
            new_score_svm = learner_svm.score(X_teste, y_teste)
            performance_history_svm.append(new_score_svm)
            clear_output(wait=True)
            print ('Accuracy SVM after query no. %d: %f' % (index+1, new_score_svm))
            arquivo_performance_svm.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_svm))
            predictions = learner_svm.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_svm.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_svm.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_svm.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_svm.write('======================================== \n')
            
    arquivo_performance_svm.write("\n Avaliação final SVM \n")
    arquivo_performance_svm.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_svm, performance_history_svm,delimiter=",")
    arquivo_performance_svm.close()

def expected_error_reduction_function_nb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import LinearSVC
    from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
    import functools
    from IPython.display import clear_output
    from copy import deepcopy
    
    #indica index da dobra para uso nos arquivos
    indica_pool=str(idx_dobra)
    
    # recupera as amostras de treino iniciais - a extratificação realizada só serve para tal finalidade.
    # No caso força a buscar pelo menos uma amostras de cada rótulo disponível (train_size= len(np.unique(y_raw)).
    # Realizar a busca aleatoriamente não garante iniciar com uma instância de cada classe.
    X_train_inicial, X_test_inicial, y_train_inicial, y_test_inicial = train_test_split(X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]], train_size= len(np.unique(y_raw[idx_data[idx_dobra][TRAIN]])) + t_inicial, stratify = y_raw[idx_data[idx_dobra][TRAIN]])
    #print("tamanho de X_train inicial: ",X_train_inicial.shape," tamanho de y_train inicial: ",y_train_inicial.shape)
    #print(y_train_inicial)  
    # recupera amostras de teste de acordo com a dobra em uso
    X_teste, y_teste = X_raw[idx_data[idx_dobra][TEST]], y_raw[idx_data[idx_dobra][TEST]]
    # recupera amostras de treino (será o pool) de acordo com a dobra em uso
    X_pool, y_pool = X_raw[idx_data[idx_dobra][TRAIN]], y_raw[idx_data[idx_dobra][TRAIN]]
    
    #isola exemplos rotulados para o treinamento inicial
    X_train = X_train_inicial
    y_train = y_train_inicial
    
    #instanciando classificadores de aprendizado ativo
    learner_nb = ActiveLearner(estimator=GaussianNB(),X_training=X_train, y_training=y_train)
    arquivo_performance_nb = open("expected_error_reduction_performance_nb_dobra_"+indica_pool+".txt","a")
    arquivo_history_nb = ("expected_error_reduction_history_nb_dobra_"+indica_pool+".csv")
        
    #verifica a performance inicial
    unqueried_score_nb = learner_nb.score(X_teste, y_teste)
    predictions = learner_nb.predict(X_teste)
    performance_history_nb.append(unqueried_score_nb)
    
      #inicio aprendizado ativo
 
    for index in range(N_QUERIES):
        n_labeled_examples_news = X_pool.shape[0]
        training_indices_news = np.random.randint(low=0, high=n_labeled_examples_news, size=BATCH_SIZE)
        amostra_recuperada_X = X_pool[training_indices_news]
        amostra_recuperada_y = y_pool[training_indices_news]
        X_pool = np.delete(X_pool, training_indices_news, axis=0)
        y_pool = np.delete(y_pool, training_indices_news, axis=0)
        if expected_error_reduction(learner_nb, X=amostra_recuperada_X) >= 0:
            #Ensina ao modelo ActiveLearner as amostras recuperadas (amostras vão para o topo).
            #learner_nb.teach(X=amostra_recuperada_X,y=amostra_recuperada_y)
            learner_nb.teach(X=X_pool[training_indices_news].reshape(BATCH_SIZE, -1),y=y_pool[training_indices_news].reshape(BATCH_SIZE, ))
            new_score_nb = learner_nb.score(X_teste, y_teste)
            performance_history_nb.append(new_score_nb)
            clear_output(wait=True)
            print ('Accuracy NB after query no. %d: %f' % (index+1, new_score_nb))
            arquivo_performance_nb.write('Accuracy after query no. %d: %f \n' % (index+1,new_score_nb))
            predictions = learner_nb.predict(X_teste)
            #print ('Precision after query no. %d: %f' % (index+1, precision_score(y_test, predictions,average='macro',zero_division=1)))
            arquivo_performance_nb.write('Precision after query no. %d: %f \n' % (index+1,precision_score(y_teste, predictions,average='macro',zero_division=1)))
            #print ('Recall after query no. %d: %f' % (index+1, recall_score(y_test, predictions, average='macro',zero_division=1)))
            arquivo_performance_nb.write('Recall after query no. %d: %f \n' % (index+1, recall_score(y_teste, predictions, average='macro',zero_division=1)))
            #print ('F1 score after query no. %d: %f' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            #arquivo_performance.write('F1 score after query no. %d: %f \n' % (index+1, f1_score(y_test, predictions,average='macro',zero_division=1)))
            f1score= 2*((precision_score(y_teste, predictions,average='macro',zero_division=1)*recall_score(y_teste, predictions, average='macro',zero_division=1))/(precision_score(y_teste, predictions,average='macro',zero_division=1)+recall_score(y_teste, predictions, average='macro',zero_division=1)))
            arquivo_performance_nb.write('F1 score after query no. %d: %f \n' % (index+1, f1score))
            #print ("========================================")
            arquivo_performance_nb.write('======================================== \n')
                      
            
    arquivo_performance_nb.write("\n Avaliação final NB \n")
    arquivo_performance_nb.write(classification_report(y_teste, predictions,zero_division=1))
    np.savetxt(arquivo_history_nb, performance_history_nb,delimiter=",")
    arquivo_performance_nb.close()

import time
import sys
import threading
from datetime import datetime
from datetime import date

from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
from modAL.uncertainty import classifier_uncertainty
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from modAL.expected_error import expected_error_reduction
from timeit import Timer
import time
import functools
# importing the multiprocessing module
import multiprocessing

#inicia relogio
t1 = time.time()
# Define o tamanho das divisões feitas no dataset (cross-validation)
n_dobras = 10
# Define Tamanho inicial da amostra (toda estratégia parte de um tamanho mínimo aleatório).
t_inicial = 10
#define array de indices das partições
idx_data =[]
# cross validation bags - n_splits
data_cv = StratifiedShuffleSplit(n_splits= n_dobras,random_state=42) 
data_cv.get_n_splits(X_raw, y_raw)
# chame a instância e gere os dados sobre a base original
type(data_cv.split(X_raw, y_raw))
# dividir os dados - A função split.split () retorna índices para amostras de treino e amostras de teste. 
# Ele examinará o número de validação cruzada especificado e retornará cada vez que treinar 
# e testar os índices de amostra usando os conjuntos de dados de treinamento e teste que podem 
# ser criados filtrando o conjunto de dados inteiro. Por exemplo idx_data[0][1], o primeiro indice faz referencia
# a dobra e o segundo indice faz referencia a posição da dobra (0 = treino e 1 = teste). Logo TRAIN=0 e TEST=1.
for train_index, test_index in data_cv.split(X_raw,y_raw):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("n_split",n_splits,"TRAIN:", train_index, "TEST:", test_index)
    idx_data.append([train_index, test_index])
#verifica tamanho das dobras (numero de instâncias de cada dobra)
#print("tamanho de cada dobra: ",idx_data[3][0].shape)


TRAIN =0
TEST =1

#inicia arrays de performance
performance_history_knn = []
performance_history_rf = []
performance_history_tree = []
performance_history_mlp = []
performance_history_xgb = []
performance_history_svm = []
performance_history_nb = []

# Define numero de queries
BATCH_SIZE = 4
N_RAW_SAMPLES = 200
N_QUERIES = N_RAW_SAMPLES // BATCH_SIZE
#chama procedimento de aprendizado para todas as dobras
for idx_dobra in range(1):
    threading.Thread(target=expected_error_reduction_function_knn(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=expected_error_reduction_function_rf(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=expected_error_reduction_function_nb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=expected_error_reduction_function_mlp(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    threading.Thread(target=expected_error_reduction_function_tree(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=expected_error_reduction_function_xgb(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()
    #threading.Thread(target=expected_error_reduction_function_svm(X_raw, y_raw, idx_data, idx_dobra, TRAIN, TEST, t_inicial)).start()

       
t2 = time.time()
time_elapsed = (t2-t1)
hours, rem = divmod(time_elapsed, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))


In [ ]:
# Plot our performance over time.
fig, ax = plt.subplots(figsize=(8.5, 6), dpi=130)

ax.plot(performance_history_knn,color="blue")
ax.plot(performance_history_rf,color="red")
ax.plot(performance_history_tree,color="green")
ax.plot(performance_history_mlp,color="yellow")
ax.plot(performance_history_xgb,color="orange")
ax.plot(performance_history_svm,color="brown")
ax.plot(performance_history_nb,color="pink")

ax.scatter(range(len(performance_history_knn)), performance_history_knn,s=0)
ax.scatter(range(len(performance_history_rf)), performance_history_rf, s=0)
ax.scatter(range(len(performance_history_tree)), performance_history_tree,s=0)
ax.scatter(range(len(performance_history_mlp)), performance_history_mlp,s=0)
ax.scatter(range(len(performance_history_xgb)), performance_history_xgb,s=0)
ax.scatter(range(len(performance_history_svm)), performance_history_svm,s=0)
ax.scatter(range(len(performance_history_nb)), performance_history_nb,s=0)

ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=5, integer=True))
ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=10))
ax.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1))

ax.set_ylim(bottom=0, top=1)
ax.grid(True)

ax.set_title('Incremental classification accuracy')
ax.set_xlabel('Query iteration')
ax.set_ylabel('Classification Accuracy')

plt.show()